In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df1=pd.read_csv(r"C:\Users\hafss\OneDrive\Desktop\PFA_2A\youtube\data\channel_stats.csv")
df1

,channelName,subscribers,views,totalVideos,playlistId
0,Ali Abdaal,6330000,493771675,1081,UUoOae5nYA7VqaXzerajD0lg


In [4]:
df2=pd.read_csv(r"C:\Users\hafss\OneDrive\Desktop\PFA_2A\youtube\data\comments.csv")
df2

,video_id,comments
0,QF2wIywvDhk,['Hope you enjoyed the video!! Get the list of...
1,-r2SH6EH5eY,['Hope you enjoyed the video!! If you fancy ge...
2,lmCaQxk4b8c,['Hope you enjoyed the video!! If you fancy ge...
3,k_K9MbqNhA0,['Hope you enjoyed the video!! If you fancy ge...
4,X_XNSeW9jok,['LifeNotes: Hope you enjoyed the video!! If y...
...,...,...
1072,FEUIxnnW5-w,"['Beautiful voice, so dreamy! 😍', '💜', 'الموسي..."
1073,d2sMEE1NrFc,"['Adorable ❤', 'Aeee :)\nWho’s watching in 202..."
1074,7APfFjfnNBc,"['buetifull voice!', 'She has such a gentle vo..."
1075,XJHM0fEH3ss,"['bruh, i thougt you were the singer but witho..."


In [ ]:
# The code itself is syntactically correct.
# The NameError arises from an issue with the PyTorch installation,
# not from a mistake in this script file.
# Please fix your PyTorch installation using the official guide.

import pandas as pd
import numpy as np
import ast
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
# The error occurs during this import:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

class YouTubeCommentsProcessor:
    """Advanced processor for YouTube comments dataset"""

    def __init__(self, csv_path=None, df=None):
        if df is not None:
            self.df = df
        elif csv_path is not None:
             # Added check for csv_path to avoid error if both are None
             self.df = pd.read_csv(csv_path)
        else:
             raise ValueError("Either csv_path or df must be provided.")

        self.processed_comments = []
        self.comment_features = []

    def parse_comments_list(self, comments_str):
        """Parse the stringified list of comments"""
        try:
            # Handle the string representation of list
            comments_list = ast.literal_eval(comments_str)
            return comments_list if isinstance(comments_list, list) else [comments_str]
        except:
            # Fallback for malformed strings or non-list strings
            # Ensure it returns a list of strings
            if isinstance(comments_str, str):
                return [comments_str]
            else:
                 return [] # Return empty list for non-string input


    def extract_comment_features(self, comment):
        """Extract rich features from individual comments"""
        features = {}

        # Basic text features
        features['length'] = len(comment)
        features['word_count'] = len(comment.split())
        features['has_url'] = 1 if any(url in comment.lower() for url in ['http', 'www.', '.com']) else 0
        features['has_mention'] = 1 if '@' in comment else 0
        # More robust emoji check (covers wider range)
        features['has_emoji'] = 1 if re.search(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639]', comment) else 0
        features['exclamation_count'] = comment.count('!')
        features['question_count'] = comment.count('?')
        features['caps_ratio'] = sum(1 for c in comment if c.isupper()) / len(comment) if len(comment) > 0 else 0

        # Sentiment indicators (can be expanded)
        positive_words = ['great', 'amazing', 'love', 'awesome', 'thank', 'good', 'best', 'excellent']
        negative_words = ['bad', 'hate', 'terrible', 'worst', 'awful', 'disappointed', 'boring']

        features['positive_word_count'] = sum(1 for word in positive_words if word in comment.lower())
        features['negative_word_count'] = sum(1 for word in negative_words if word in comment.lower())

        # Topic indicators based on your data (can be expanded)
        productivity_keywords = ['productivity', 'focus', 'work', 'time', 'efficient', 'procrastinate']
        finance_keywords = ['money', 'rich', 'wealth', 'invest', 'debt', 'financial', 'million']
        motivation_keywords = ['inspire', 'motivate', 'dream', 'goal', 'success', 'achieve']

        features['productivity_score'] = sum(1 for word in productivity_keywords if word in comment.lower())
        features['finance_score'] = sum(1 for word in finance_keywords if word in comment.lower())
        features['motivation_score'] = sum(1 for word in motivation_keywords if word in comment.lower())

        return features

    def classify_comment_type(self, comment):
        """Classify comments into categories based on content analysis"""
        comment_lower = comment.lower()

        # Define classification rules based on your actual data patterns
        # Ensure 'black algorithm' is split or checked carefully if it appears as one token
        if any(spam_indicator in comment_lower for spam_indicator in ['black algorithm', 'betterhelp', 'lois e wilson', 'check out my channel', 'subscribe', 'link in bio']): # Added more spam indicators
            return 'spam_promotional'
        elif any(gratitude in comment_lower for gratitude in ['thank', 'grateful', 'appreciate', 'thanks']): # Added 'thanks'
            return 'gratitude'
        elif any(question in comment for question in ['?', 'how', 'what', 'where', 'when', 'why', 'can you', 'could you']): # Added more question starters
            return 'question'
        elif any(criticism in comment_lower for criticism in ['disappointed', 'lazy', 'disagree', 'wrong', 'bad', 'terrible', 'hate']): # Added more criticism terms
            return 'criticism'
        elif any(personal in comment_lower for personal in ['i am', 'my', 'personally', 'i have', 'i feel', 'in my opinion', 'my experience']): # Added more personal indicators
            return 'personal_story'
        elif len(comment) > 200: # Increased threshold for detailed feedback
            return 'detailed_feedback'
        else:
            return 'general_comment'

    def process_dataset(self):
        """Process the entire dataset and extract features"""
        all_comments = []
        # video_comment_mapping = [] # This variable is created but not used

        # Check if df is loaded
        if self.df is None or self.df.empty:
             print("DataFrame is empty or not loaded. Cannot process.")
             self.processed_df = pd.DataFrame() # Initialize empty df
             return self.processed_df

        for idx, row in self.df.iterrows():
            # Ensure required columns exist
            if 'video_id' not in row or 'comments' not in row:
                print(f"Skipping row {idx}: Missing 'video_id' or 'comments' column.")
                continue

            video_id = row['video_id']
            comments_list = self.parse_comments_list(row['comments'])

            for i, comment in enumerate(comments_list):
                # Ensure comment is a non-empty string before processing
                if isinstance(comment, str) and len(comment.strip()) > 0:
                    comment_stripped = comment.strip() # Use stripped comment

                    # Extract features
                    features = self.extract_comment_features(comment_stripped)
                    comment_type = self.classify_comment_type(comment_stripped)

                    comment_data = {
                        'video_id': video_id,
                        'comment_position': i,
                        'comment_text': comment_stripped,
                        'comment_type': comment_type,
                        **features
                    }

                    all_comments.append(comment_data)
                    # video_comment_mapping.append((video_id, i)) # Not used

        self.processed_df = pd.DataFrame(all_comments)
        return self.processed_df

class CommentDataset(Dataset):
    """Custom dataset for comment analysis"""

    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Ensure text is a string, even if pandas returns something else sometimes
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            # Ensure labels are handled correctly, assuming they are integers
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

class MultiTaskCommentAnalyzer(nn.Module):
    """Advanced multi-task model for comment analysis"""

    # Increased default num_classes as classify_comment_type now has 7 categories
    def __init__(self, model_name='distilbert-base-uncased', num_classes=7, num_features=15):
        super().__init__()

        # BERT-based encoder
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)

        # Feature fusion layer - Check if features will always be provided
        # Added a check or flexibility if features are not used
        self.num_features = num_features # Store num_features
        if num_features > 0:
            self.feature_projection = nn.Linear(num_features, 128)
            combined_size = self.bert.config.hidden_size + 128
        else:
            combined_size = self.bert.config.hidden_size
            self.feature_projection = None # No projection needed


        # Task-specific heads
        # Use slightly different layer sizes for each task
        self.comment_type_classifier = nn.Sequential(
            nn.Linear(combined_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3), # Increased dropout
            nn.Linear(512, num_classes)
        )

        self.sentiment_classifier = nn.Sequential(
            nn.Linear(combined_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3), # Increased dropout
            nn.Linear(256, 3)  # positive, negative, neutral (Needs labels for this task)
        )

        # Engagement predictor (Needs labels for this task, e.g., likes/replies count)
        self.engagement_predictor = nn.Sequential(
            nn.Linear(combined_size, 128), # Smaller layer
            nn.ReLU(),
            nn.Dropout(0.3), # Increased dropout
            nn.Linear(128, 1) # Predicting a single continuous value (like count, reply count)
        )

        # Next comment topic predictor - This is more complex and usually requires sequence data
        # The current implementation isn't suitable for the standard forward pass on independent comments.
        # It would need a dedicated sequence model (like LSTM on comment embeddings over time).
        # For this multi-task model operating on individual comments, the topic predictor isn't quite right here.
        # Let's remove it from the standard forward pass and explain this.
        # self.topic_predictor_lstm = nn.LSTM(combined_size, 256, batch_first=True, bidirectional=True)
        # self.topic_output_linear = nn.Linear(512, num_classes) # Predicting next topic class

        # Note: The 'next_topic' prediction part in this model's forward pass is commented out
        # as it requires sequential input (a series of comments for a video),
        # whereas the main forward pass is designed for processing individual comments.
        # A separate sequence model or a modified forward pass handling batches of sequences would be needed.


    def forward(self, input_ids, attention_mask, features=None): # Removed sequence_mode flag
        # BERT encoding
        # Ensure bert_output is a tuple/object with pooler_output attribute
        # Different BERT models might return different output types
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Use the pooled output (usually the [CLS] token representation)
        if hasattr(bert_output, 'pooler_output') and bert_output.pooler_output is not None:
             pooled_output = bert_output.pooler_output
        # Fallback to the first token representation if pooler_output is None (e.g., some models)
        elif hasattr(bert_output, 'last_hidden_state'):
             pooled_output = bert_output.last_hidden_state[:, 0] # Take the [CLS] token output
        else:
             raise AttributeError("BERT model output does not have 'pooler_output' or 'last_hidden_state'.")

        pooled_output = self.dropout(pooled_output)

        # Feature fusion if provided and feature_projection exists
        if features is not None and self.feature_projection is not None:
            # Ensure features is the correct dtype (float)
            features = features.float()
            feature_embedding = self.feature_projection(features)
            combined_features = torch.cat([pooled_output, feature_embedding], dim=1)
        else:
            combined_features = pooled_output

        # Task outputs
        comment_type_logits = self.comment_type_classifier(combined_features)
        # Sentiment and Engagement outputs would require corresponding labels in the dataset
        # For this example, we'll focus on the comment type classification
        # sentiment_logits = self.sentiment_classifier(combined_features)
        # engagement_score = self.engagement_predictor(combined_features)

        # Return just the comment type for the main task
        return {'comment_type': comment_type_logits}
        # If training all tasks, return all:
        # return {
        #     'comment_type': comment_type_logits,
        #     'sentiment': sentiment_logits,
        #     'engagement': engagement_score
        # }


class NextCommentPredictor:
    """Specialized class for predicting next comment characteristics"""

    def __init__(self, processed_df):
        self.df = processed_df
        self.video_sequences = {}
        self.build_sequences()
        # Initialize LabelEncoder for topics if needed
        self.topic_encoder = LabelEncoder()
        if not self.df.empty:
            # Fit encoder on all unique comment types
            self.topic_encoder.fit(self.df['comment_type'].unique())
            self.num_topics = len(self.topic_encoder.classes_)
        else:
            self.num_topics = 0

    def build_sequences(self):
        """Build comment sequences for each video"""
        if self.df.empty:
            print("DataFrame is empty. Cannot build sequences.")
            return
        for video_id in self.df['video_id'].unique():
            video_comments = self.df[self.df['video_id'] == video_id].sort_values('comment_position')
            # Ensure the sequence has comment types
            if 'comment_type' in video_comments.columns and not video_comments.empty:
                 self.video_sequences[video_id] = video_comments
            else:
                 print(f"Video {video_id} has no comments or no 'comment_type' column.")


    def create_sequence_dataset(self, sequence_length=5):
        """Create sequences for training next comment prediction"""
        sequences = [] # Store encoded sequences of comment types
        targets = []   # Store encoded target comment types

        if not self.video_sequences:
             print("No video sequences available.")
             return np.array([]), np.array([]) # Return empty arrays

        for video_id, comments in self.video_sequences.items():
            # Ensure comments DataFrame is not empty and has 'comment_type'
            if comments.empty or 'comment_type' not in comments.columns:
                continue

            try:
                # Encode comment types to numerical labels
                encoded_comment_types = self.topic_encoder.transform(comments['comment_type'].values)
            except ValueError as e:
                 print(f"Error encoding comment types for video {video_id}: {e}. Skipping.")
                 continue


            if len(encoded_comment_types) < sequence_length + 1:
                continue

            for i in range(len(encoded_comment_types) - sequence_length):
                seq = encoded_comment_types[i:i+sequence_length]
                target = encoded_comment_types[i+sequence_length]
                sequences.append(seq)
                targets.append(target)

        return np.array(sequences), np.array(targets)

    # This method is for *analyzing* existing sequences, not predicting with the model
    def predict_next_based_on_frequency(self, sequence_prefix):
        """
        Simple frequency-based prediction of the next topic
        given a sequence prefix of comment types.
        (This is a baseline, not using the neural network)
        """
        if self.df.empty or self.num_topics == 0:
             print("DataFrame empty or topics not encoded.")
             return None

        # Encode the sequence prefix
        try:
            encoded_prefix = self.topic_encoder.transform(sequence_prefix)
        except ValueError as e:
            print(f"Error encoding sequence prefix: {e}")
            return None


        # Find all sequences in the dataset that start with this prefix
        matching_targets = []
        sequences, targets = self.create_sequence_dataset(sequence_length=len(encoded_prefix))

        if sequences.size == 0: # Check if create_sequence_dataset returned empty arrays
            print("No sequences generated from dataset.")
            return None

        # Convert sequences to tuples for easy comparison
        sequence_tuples = [tuple(s) for s in sequences]
        encoded_prefix_tuple = tuple(encoded_prefix)

        # Collect targets that immediately follow the prefix
        for i in range(len(sequence_tuples)):
            # Check if the sequence starts with the prefix
            if sequence_tuples[i][:len(encoded_prefix_tuple)] == encoded_prefix_tuple:
                 # The target for this sequence *is* the element immediately following the prefix
                 # in the original dataset's sequences where the prefix appeared.
                 # We need to find the element *after* the sequence of length len(encoded_prefix_tuple)
                 # However, the create_sequence_dataset method already aligns targets correctly.
                 # The target 'targets[i]' is the comment type *after* the sequence 'sequences[i]'.
                 # So we just need to match sequences[i] exactly to our encoded_prefix.
                 # A more accurate method would be to look at sequences of length N+1 and find where the first N elements match.
                 # Let's adjust create_sequence_dataset or this logic.

                 # Simpler approach: Iterate through original video sequences
                 # This is less efficient but conceptually matches finding the topic after a prefix.
                 pass # We'll implement this below

        # More efficient prefix matching directly on original sequences data
        next_topics_after_prefix = []
        prefix_len = len(encoded_prefix)

        for video_id, comments in self.video_sequences.items():
            if 'comment_type' not in comments.columns: continue

            try:
                encoded_video_sequence = self.topic_encoder.transform(comments['comment_type'].values)
            except ValueError:
                continue # Skip if encoding fails

            for i in range(len(encoded_video_sequence) - prefix_len):
                if np.array_equal(encoded_video_sequence[i : i + prefix_len], encoded_prefix):
                    if i + prefix_len < len(encoded_video_sequence):
                        next_topics_after_prefix.append(encoded_video_sequence[i + prefix_len])

        if not next_topics_after_prefix:
            print(f"No occurrences found for sequence prefix: {sequence_prefix}")
            # Fallback: return the overall most frequent topic
            if self.num_topics > 0:
                 overall_topics = self.df['comment_type'].unique()
                 overall_counts = self.df['comment_type'].value_counts()
                 most_common_overall = overall_counts.index[0]
                 print(f"Returning overall most frequent topic: {most_common_overall}")
                 return most_common_overall
            else:
                 return None # Cannot fallback


        # Count occurrences of topics that follow the prefix
        next_topic_counts = Counter(next_topics_after_prefix)

        if not next_topic_counts:
             print("No topics found following the sequence prefix.")
             return None

        # Get the most common topic label (encoded)
        most_likely_next_topic_encoded = next_topic_counts.most_common(1)[0][0]

        # Decode the label
        most_likely_next_topic = self.topic_encoder.inverse_transform([most_likely_next_topic_encoded])[0]

        return most_likely_next_topic


class CommentAnalysisPipeline:
    """Complete pipeline for comment analysis and prediction"""

    def __init__(self, data_path=None, df=None):
        self.processor = YouTubeCommentsProcessor(csv_path=data_path, df=df)
        # Consider adding 'do_lower_case=True' if needed based on model pre-training
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        self.label_encoder = LabelEncoder()
        self.model = None
        self.processed_df = None # Initialize processed_df
        self.feature_columns = [] # To store feature column names
        self.predictor = None # Initialize predictor

    def prepare_data(self):
        """Prepare and process the dataset"""
        print("Processing comments dataset...")
        self.processed_df = self.processor.process_dataset()

        if self.processed_df.empty:
            print("Processing resulted in an empty DataFrame. Cannot proceed.")
            return self.processed_df

        print(f"Total comments processed: {len(self.processed_df)}")
        print(f"Comment types distribution:")
        print(self.processed_df['comment_type'].value_counts())

        # Identify feature columns after processing
        self.feature_columns = [col for col in self.processed_df.columns
                               if col not in ['video_id', 'comment_position', 'comment_text', 'comment_type']]

        # Initialize the sequence predictor now that processed_df is ready
        self.predictor = NextCommentPredictor(self.processed_df)

        return self.processed_df

    def train_model(self, test_size=0.2, batch_size=16, epochs=3):
        """Train the multi-task model (currently only comment type)"""
        if self.processed_df is None or self.processed_df.empty:
            print("Data not processed. Run prepare_data() first.")
            return None

        # Prepare features
        # Ensure feature columns exist in the processed df
        available_feature_cols = [col for col in self.feature_columns if col in self.processed_df.columns]

        if not available_feature_cols:
             print("Warning: No feature columns found for training.")
             X_features = np.zeros((len(self.processed_df), 0)) # Empty feature array
             num_features = 0
        else:
             X_features = self.processed_df[available_feature_cols].values
             # Handle potential non-numeric data in features (shouldn't happen with current features, but good practice)
             # X_features = X_features.astype(float) # Ensure float type
             num_features = X_features.shape[1]


        X_text = self.processed_df['comment_text'].values
        y = self.label_encoder.fit_transform(self.processed_df['comment_type'].values)

        # Check if there are enough samples for splitting
        if len(X_text) < 2:
             print("Not enough data samples to perform train-test split.")
             return None
        if len(np.unique(y)) < 2:
             print("Only one class found. Cannot perform classification training.")
             return None
        if len(np.unique(y)) < 2 / test_size: # Heuristic check for meaningful stratification
             print(f"Warning: Very few samples per class for test_size={test_size}. Stratification might be inaccurate.")


        # Train-test split
        # Stratify requires at least one sample per class in both train and test sets.
        # If a class has only 1 sample, stratification will fail.
        try:
            X_text_train, X_text_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
                X_text, X_features, y, test_size=test_size, random_state=42, stratify=y
            )
        except ValueError as e:
             print(f"Error during train-test split (stratify failed): {e}")
             print("Try reducing test_size or handle classes with very few samples.")
             # Fallback to non-stratified split or skip training
             print("Attempting non-stratified split...")
             try:
                  X_text_train, X_text_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
                      X_text, X_features, y, test_size=test_size, random_state=42
                  )
                  print("Non-stratified split successful.")
             except Exception as non_strat_e:
                  print(f"Non-stratified split also failed: {non_strat_e}")
                  return None # Cannot train if split fails


        # Create datasets
        train_dataset = CommentDataset(X_text_train, y_train, self.tokenizer)
        test_dataset = CommentDataset(X_text_test, y_test, self.tokenizer)

        # Use the corresponding features for the dataset
        # Note: The current CommentDataset only takes texts and labels, not features.
        # To train with features, the Dataset class needs to be modified or features passed separately to the DataLoader.
        # For now, the model's forward expects features=None unless passed.
        # Let's modify the Dataset and DataLoader to handle features.

        class CommentDatasetWithFeatures(Dataset):
            def __init__(self, texts, features, labels, tokenizer, max_length=128):
                self.texts = texts
                self.features = features
                self.labels = labels
                self.tokenizer = tokenizer
                self.max_length = max_length

            def __len__(self):
                return len(self.texts)

            def __getitem__(self, idx):
                text = str(self.texts[idx])
                feature_row = self.features[idx]

                encoding = self.tokenizer(
                    text,
                    truncation=True,
                    padding='max_length',
                    max_length=self.max_length,
                    return_tensors='pt'
                )

                return {
                    'input_ids': encoding['input_ids'].flatten(),
                    'attention_mask': encoding['attention_mask'].flatten(),
                    'features': torch.tensor(feature_row, dtype=torch.float), # Ensure float type
                    'labels': torch.tensor(self.labels[idx], dtype=torch.long)
                }

        train_dataset = CommentDatasetWithFeatures(X_text_train, X_feat_train, y_train, self.tokenizer)
        test_dataset = CommentDatasetWithFeatures(X_text_test, X_feat_test, y_test, self.tokenizer)


        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)

        # Initialize model
        num_classes = len(self.label_encoder.classes_)
        self.model = MultiTaskCommentAnalyzer(num_classes=num_classes, num_features=num_features)

        # Set device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        print(f"Using device: {device}")

        # Training setup
        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        # Only using CrossEntropyLoss for the comment type task for now
        criterion = nn.CrossEntropyLoss()

        # Training loop
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch in train_loader:
                optimizer.zero_grad()

                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                features = batch['features'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    features=features # Pass features to the model
                )

                # Calculate loss for the main task (comment_type)
                loss = criterion(outputs['comment_type'], labels)

                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs}, Average Loss: {total_loss/len(train_loader):.4f}")

        # Evaluation
        print("\nEvaluating model on test set...")
        self.evaluate_model(test_loader, device) # Pass device to evaluation

        return self.model

    def evaluate_model(self, test_loader, device):
        """Evaluate the trained model"""
        if self.model is None:
             print("Model not trained yet.")
             return [], []
        if self.processed_df.empty:
             print("Data not processed. Cannot evaluate.")
             return [], []


        self.model.eval()
        predictions = []
        true_labels = []
        all_preds = [] # Store raw predictions for confidence analysis etc.

        with torch.no_grad():
            for batch in test_loader:
                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                features = batch['features'].to(device) # Pass features
                labels = batch['labels'].to(device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    features=features # Pass features
                )

                # Assuming comment_type is the primary task evaluation
                logits = outputs['comment_type']
                preds = torch.argmax(logits, dim=1)

                predictions.extend(preds.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())
                all_preds.extend(logits.cpu().numpy()) # Store raw logits


        # Classification report
        if hasattr(self, 'label_encoder') and self.label_encoder is not None:
             class_names = self.label_encoder.classes_
             print("\nClassification Report (Comment Type):")
             try:
                print(classification_report(true_labels, predictions, target_names=class_names, zero_division=0))
             except ValueError as e:
                 print(f"Could not generate classification report: {e}")
                 print("This can happen if true_labels or predictions contain classes not in target_names.")
                 # Fallback or print raw arrays for debugging
                 # print("True labels:", true_labels)
                 # print("Predictions:", predictions)
                 # print("Class names:", class_names)

        else:
             print("LabelEncoder not initialized. Cannot print class names.")
             print("\nClassification Report (Comment Type):")
             print(classification_report(true_labels, predictions, zero_division=0))


        # Confusion Matrix (Optional but helpful visualization)
        # try:
        #     cm = confusion_matrix(true_labels, predictions)
        #     plt.figure(figsize=(10, 8))
        #     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        #     plt.xlabel('Predicted Label')
        #     plt.ylabel('True Label')
        #     plt.title('Confusion Matrix (Comment Type)')
        #     plt.show()
        # except Exception as e:
        #     print(f"Could not generate confusion matrix plot: {e}")


        return predictions, true_labels

    def predict_next_comment_topic(self, video_id, current_sequence_comments, sequence_length=5):
        """
        Predict the topic of the next comment given a sequence of recent comments
        for a specific video using the frequency-based predictor.

        Args:
            video_id (str): The ID of the video.
            current_sequence_comments (list): A list of string comments forming the sequence.
            sequence_length (int): The length of the sequence to consider (for simple frequency).
                                   Note: The frequency predictor uses the *exact* provided sequence length.

        Returns:
            str: The predicted topic of the next comment, or None if prediction fails.
        """
        if self.predictor is None:
            print("Predictor not initialized. Run prepare_data() first.")
            return None

        if not current_sequence_comments:
             print("Current sequence is empty. Cannot predict.")
             return None

        # Use only the last 'sequence_length' comments if the provided list is longer
        sequence_to_use = current_sequence_comments[-sequence_length:]

        print(f"Predicting next topic based on sequence: {sequence_to_use}")

        # The NextCommentPredictor requires the sequence of *types*, not text.
        # We need to classify the types of the input comments first.
        predicted_types_sequence = [self.processor.classify_comment_type(comment) for comment in sequence_to_use]

        # Now use the frequency predictor with the sequence of predicted types
        predicted_next_topic = self.predictor.predict_next_based_on_frequency(predicted_types_sequence)

        if predicted_next_topic:
             print(f"Predicted next topic: {predicted_next_topic}")
        else:
             print("Prediction failed or no matching sequence found.")

        return predicted_next_topic


    def analyze_comment_patterns(self):
        """Analyze patterns in the comment data"""
        if self.processed_df is None or self.processed_df.empty:
            print("Data not processed. Run prepare_data() first.")
            return

        print("\n=== Comment Analysis Report ===")
        print("=" * 30)

        # Basic statistics
        print(f"Total videos: {self.processed_df['video_id'].nunique()}")
        print(f"Total comments: {len(self.processed_df)}")
        if self.processed_df['video_id'].nunique() > 0:
             print(f"Average comments per video: {len(self.processed_df) / self.processed_df['video_id'].nunique():.2f}")
        else:
             print("Average comments per video: N/A (No videos found)")

        # Comment type distribution
        print("\nComment Type Distribution:")
        type_counts = self.processed_df['comment_type'].value_counts()
        if not type_counts.empty:
            total_comments = len(self.processed_df)
            for comment_type, count in type_counts.items():
                print(f"  {comment_type}: {count} ({count/total_comments*100:.1f}%)")
            # Optional: Plot distribution
            # plt.figure(figsize=(10, 6))
            # sns.countplot(data=self.processed_df, y='comment_type', order=type_counts.index, palette='viridis')
            # plt.title('Distribution of Comment Types')
            # plt.xlabel('Count')
            # plt.ylabel('Comment Type')
            # plt.show()
        else:
             print("  No comment types found.")


        # Length analysis
        print(f"\nComment Length Statistics:")
        if not self.processed_df.empty and 'length' in self.processed_df.columns:
             print(f"  Average length: {self.processed_df['length'].mean():.2f} characters")
             print(f"  Median length: {self.processed_df['length'].median():.2f} characters")
             # Optional: Plot distribution
             # plt.figure(figsize=(8, 4))
             # sns.histplot(self.processed_df['length'], bins=50, kde=True)
             # plt.title('Distribution of Comment Lengths')
             # plt.xlabel('Length (characters)')
             # plt.ylabel('Frequency')
             # plt.show()
        else:
             print("  Length statistics not available.")


        # Topic analysis (based on keyword scores)
        print(f"\nTopic Keyword Analysis:")
        if not self.processed_df.empty:
            prod_count = (self.processed_df['productivity_score'] > 0).sum() if 'productivity_score' in self.processed_df.columns else 0
            fin_count = (self.processed_df['finance_score'] > 0).sum() if 'finance_score' in self.processed_df.columns else 0
            mot_count = (self.processed_df['motivation_score'] > 0).sum() if 'motivation_score' in self.processed_df.columns else 0

            print(f"  Productivity-related (score > 0): {prod_count} comments")
            print(f"  Finance-related (score > 0): {fin_count} comments")
            print(f"  Motivation-related (score > 0): {mot_count} comments")
        else:
             print("  Topic keyword analysis not available.")

        print("=" * 30)


# Example usage and execution
def main():
    """Main execution function"""
    print("🚀 Starting YouTube Comments AI Analysis Pipeline")
    print("=" * 50)

    # --- DEMO SETUP ---
    # For demonstration, creating sample structure similar to expected input
    # In a real scenario, you would load your data from a CSV.
    # Example using a simple CSV file structure:
    # video_id,comments
    # video_a,['Great video!', 'Thanks for this.', 'How can I do X?']
    # video_b,["Amazing content!", "I disagree.", "This is really bad."]
    # video_c,"['Personal story here...', 'Another comment', 'spam_indicator like betterhelp']"


    # Create a dummy CSV file for demonstration
    dummy_csv_path = "dummy_comments.csv"
    dummy_data_content = """video_id,comments
video_1,"['Great video, thanks for sharing!', 'This helped me a lot with my productivity.', 'Question: How did you edit this part?', 'I hate this part.', 'check out my channel for similar content']"
video_2,"['Amazing content!', 'What camera do you use?', 'My personal experience was different.', 'Thank you so much!', 'This is the best!', 'another great comment']"
video_3,"['Short comment.', 'Another short comment!', 'Detailed feedback: This video was very informative, but I think you could improve the audio quality. The topic on finance was particularly interesting, but it could be more in-depth. Overall good though.']"
video_4,"['Spam link www.mysite.com', 'Lois E Wilson is a scammer', 'Black algorithm problems']"
"""
    try:
        with open(dummy_csv_path, "w") as f:
            f.write(dummy_data_content)
        print(f"Created dummy data file: {dummy_csv_path}")

        # Initialize pipeline using the dummy CSV path
        pipeline = CommentAnalysisPipeline(data_path=dummy_csv_path)

        # Process data
        processed_df = pipeline.prepare_data()

        if not processed_df.empty:
            # Analyze patterns
            pipeline.analyze_comment_patterns()

            # --- MODEL TRAINING (Optional for analysis, requires sufficient data) ---
            # Note: Training a BERT-based model requires significant resources (RAM, potentially GPU)
            # and a substantial amount of labeled data for meaningful results.
            # The dummy data is too small for effective training.
            # This part is commented out for demonstration with small dummy data.

            # print("\n🔬 Training Model (using dummy data - results may not be meaningful)...")
            # # You might need more epochs and a larger batch size for real data
            # # Ensure you have enough data for a split (at least 2 samples)
            # if len(processed_df) > 10: # Simple check if there's *some* data
            #     model = pipeline.train_model(epochs=1, batch_size=8) # Use smaller batch/epochs for demo
            #     if model:
            #         print("\nModel training complete.")
            #     else:
            #         print("\nModel training skipped due to data issues.")
            # else:
            #      print("\nSkipping model training: Not enough data in processed_df.")


            # --- NEXT COMMENT PREDICTION DEMO (using frequency baseline) ---
            print("\n🔮 Demonstrating Next Comment Prediction (Frequency Baseline)")
            # Example sequence from video_1: 'Great video, thanks for sharing!', 'This helped me a lot with my productivity.'
            video_id_to_predict = 'video_1'
            # Find the sequence of comments from the processed_df for video_1
            video_1_comments_df = processed_df[processed_df['video_id'] == video_id_to_predict].sort_values('comment_position')

            if not video_1_comments_df.empty:
                 # Use the first 2 comments as the sequence prefix
                 demo_sequence = video_1_comments_df['comment_text'].tolist()[:2]
                 if len(demo_sequence) == 2:
                     print(f"\nPredicting next topic for video '{video_id_to_predict}' based on sequence: {demo_sequence}")
                     predicted_topic = pipeline.predict_next_comment_topic(video_id_to_predict, demo_sequence)
                     if predicted_topic:
                         print(f"Predicted next topic type (based on frequency): {predicted_topic}")
                     else:
                         print("Prediction failed.")
                 else:
                     print(f"Not enough comments in video '{video_id_to_predict}' to form a sequence of length 2.")
            else:
                 print(f"No comments found for video '{video_id_to_predict}' in processed data.")


        else:
            print("\nPipeline stopped because processed data is empty.")

    except FileNotFoundError:
        print(f"Error: Dummy data file not found at {dummy_csv_path}")
        print("Please ensure the script can create/access this file, or provide your own data.")
    except Exception as e:
        print(f"An unexpected error occurred during the main execution: {e}")

    print("\n✅ Pipeline setup complete (execution finished).")
    print("Remember to fix your PyTorch installation if you encountered the NameError.")
    print("To use with your own CSV data:")
    print("1. Ensure your CSV has 'video_id' and 'comments' columns.")
    print("2. Initialize: pipeline = CommentAnalysisPipeline(data_path='path/to/your/file.csv')")
    print("3. Process: processed_df = pipeline.prepare_data()")
    print("4. Train (Optional, needs data): model = pipeline.train_model()")
    print("5. Analyze: pipeline.analyze_comment_patterns()")
    print("6. Predict Next Topic (Optional): pipeline.predict_next_comment_topic('your_video_id', ['comment1', 'comment2'])")


if __name__ == "__main__":
    main()

NameError: name '_C' is not defined

In [ ]:
pip uninstall torch torchaudio torchvision

In [5]:
df3=pd.read_csv(r"C:\Users\hafss\OneDrive\Desktop\PFA_2A\youtube\data\video_details_raw.csv")
df3

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,QF2wIywvDhk,Ali Abdaal,I Tried AI as a Life Coach for 365 Days - Here...,"#ad Going to therapy is a sign of strength, no...","['Ali Abdaal', 'Ali abdal']",2025-05-31T13:00:44Z,29534,1323,NaN,145,PT36M42S,hd,False
1,-r2SH6EH5eY,Ali Abdaal,Getting Rich is a Game. Here’s How to Win.,Sign up and download Grammarly for FREE. If yo...,"['Ali Abdaal', 'Ali abdal']",2025-05-28T13:30:01Z,101554,4581,NaN,312,PT43M12S,hd,False
2,lmCaQxk4b8c,Ali Abdaal,Here's How to Get to the Top 1% (Discipline Is...,Check out Spotter to brainstorm better video i...,"['Ali Abdaal', 'Ali abdal', 'Joe Hudson']",2025-05-09T16:23:47Z,305074,11357,NaN,609,PT1H9M34S,hd,False
3,k_K9MbqNhA0,Ali Abdaal,17 Cheap Purchases That Actually Improved My Life,Check out DeleteMe using my link and get 20% o...,"['Ali Abdaal', 'Ali abdal']",2025-05-03T13:01:19Z,191858,5755,NaN,360,PT16M25S,hd,False
4,X_XNSeW9jok,Ali Abdaal,How To Get Rich,"💌 Join LifeNotes, my weekly email where I shar...","['Ali Abdaal', 'Ali abdal']",2025-04-18T15:00:51Z,645630,29902,NaN,1795,PT28M7S,hd,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,FEUIxnnW5-w,Ali Abdaal,When I Was Your Man (Bruno Mars) - Duranka Per...,Duranka came over to Cambridge last night to w...,NaN,2017-01-28T18:14:19Z,23920,306,NaN,24,PT3M33S,hd,False
1074,d2sMEE1NrFc,Ali Abdaal,Thinking Out Loud (Ed Sheeran) - Ali Abdaal & ...,Another (quite bassy) cover of one of our favo...,"['thinking out loud', 'ed sheeran', 'acoustic ...",2017-01-18T07:13:58Z,8736,193,NaN,46,PT3M42S,hd,False
1075,7APfFjfnNBc,Ali Abdaal,Payphone (Maroon 5) - Katherine Macfarland & A...,"First cover of 2017! We both love this song, s...","['payphone', 'maroon 5', 'acoustic cover', 'co...",2017-01-04T23:13:51Z,103826,2429,NaN,162,PT2M35S,hd,False
1076,XJHM0fEH3ss,Ali Abdaal,Rolling in the Deep (Adele) - Duranka Perera &...,Our first 'proper' video! One of our all-time ...,"['rolling in the deep', 'adele', 'cover', 'aco...",2016-04-05T09:05:29Z,72610,857,NaN,48,PT2M29S,hd,False


In [17]:
df=pd.read_csv(r"C:\Users\hafss\OneDrive\Desktop\PFA_2A\youtube\data\video_details.csv")
df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,publishDayName,durationSecs,tagCount
0,QF2wIywvDhk,Ali Abdaal,I Tried AI as a Life Coach for 365 Days - Here...,"#ad Going to therapy is a sign of strength, no...","['Ali Abdaal', 'Ali abdal']",2025-05-31 13:00:44+00:00,29534,1323,NaN,145,PT36M42S,hd,False,Saturday,2202.0,0
1,-r2SH6EH5eY,Ali Abdaal,Getting Rich is a Game. Here’s How to Win.,Sign up and download Grammarly for FREE. If yo...,"['Ali Abdaal', 'Ali abdal']",2025-05-28 13:30:01+00:00,101554,4581,NaN,312,PT43M12S,hd,False,Wednesday,2592.0,0
2,lmCaQxk4b8c,Ali Abdaal,Here's How to Get to the Top 1% (Discipline Is...,Check out Spotter to brainstorm better video i...,"['Ali Abdaal', 'Ali abdal', 'Joe Hudson']",2025-05-09 16:23:47+00:00,305074,11357,NaN,609,PT1H9M34S,hd,False,Friday,4174.0,0
3,k_K9MbqNhA0,Ali Abdaal,17 Cheap Purchases That Actually Improved My Life,Check out DeleteMe using my link and get 20% o...,"['Ali Abdaal', 'Ali abdal']",2025-05-03 13:01:19+00:00,191858,5755,NaN,360,PT16M25S,hd,False,Saturday,985.0,0
4,X_XNSeW9jok,Ali Abdaal,How To Get Rich,"💌 Join LifeNotes, my weekly email where I shar...","['Ali Abdaal', 'Ali abdal']",2025-04-18 15:00:51+00:00,645630,29902,NaN,1795,PT28M7S,hd,False,Friday,1687.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,FEUIxnnW5-w,Ali Abdaal,When I Was Your Man (Bruno Mars) - Duranka Per...,Duranka came over to Cambridge last night to w...,NaN,2017-01-28 18:14:19+00:00,23920,306,NaN,24,PT3M33S,hd,False,Saturday,213.0,0
1074,d2sMEE1NrFc,Ali Abdaal,Thinking Out Loud (Ed Sheeran) - Ali Abdaal & ...,Another (quite bassy) cover of one of our favo...,"['thinking out loud', 'ed sheeran', 'acoustic ...",2017-01-18 07:13:58+00:00,8736,193,NaN,46,PT3M42S,hd,False,Wednesday,222.0,0
1075,7APfFjfnNBc,Ali Abdaal,Payphone (Maroon 5) - Katherine Macfarland & A...,"First cover of 2017! We both love this song, s...","['payphone', 'maroon 5', 'acoustic cover', 'co...",2017-01-04 23:13:51+00:00,103826,2429,NaN,162,PT2M35S,hd,False,Wednesday,155.0,0
1076,XJHM0fEH3ss,Ali Abdaal,Rolling in the Deep (Adele) - Duranka Perera &...,Our first 'proper' video! One of our all-time ...,"['rolling in the deep', 'adele', 'cover', 'aco...",2016-04-05 09:05:29+00:00,72610,857,NaN,48,PT2M29S,hd,False,Tuesday,149.0,0


In [15]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'publishedAt' is a datetime column
# If it's still a string, parse it first:
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True, errors='coerce')

# Now create the time-based features
df['publish_hour'] = df['publishedAt'].dt.hour
df['publish_day_of_week'] = df['publishedAt'].dt.dayofweek  # Monday=0, Sunday=6
df['publish_month'] = df['publishedAt'].dt.month

# Days since publish
df['days_since_publish'] = (pd.Timestamp.now(tz='UTC') - df['publishedAt']).dt.days


In [16]:
df

,ideo_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,publishDayName,durationSecs,tagCount,publish_hour,publish_day_of_week,publish_month,days_since_publish
0,id1,Channel A,Video Title 1: How to start,Desc 1 about starting.,"['tag1', 'tag2']",2023-01-01 10:00:00+00:00,15000,800,0,50,PT10M5S,hd,False,None,None,None,10,6,1,881
1,id2,Channel B,Video Title 2 Explained,Desc 2 explains.,['tag3'],2023-01-05 12:30:00+00:00,50000,3000,0,150,PT20M30S,hd,True,None,None,None,12,3,1,877
2,id3,Channel A,Another Video Idea,Desc 3 has ideas.,"['tag1', 'tag4']",2023-01-10 15:00:00+00:00,20000,1000,0,75,PT15M0S,sd,False,None,None,None,15,1,1,872
3,id4,Channel C,Quick Tip #1,Short desc for quick tip.,[],2023-02-01 08:00:00+00:00,5000,200,0,10,PT5M10S,hd,False,None,None,None,8,2,2,850
4,id5,Channel B,Long Tutorial on Topic Y,Very long description for tutorial...,"['tag5', 'tag6', 'tag7']",2023-02-15 20:00:00+00:00,75000,4000,0,250,PT45M0S,hd,True,None,None,None,20,2,2,836
5,id6,Channel A,Short explainer video,Brief explanation.,['tag1'],2023-03-01 11:00:00+00:00,18000,900,0,60,PT8M30S,hd,False,None,None,None,11,2,3,822
6,id7,Channel C,Quick Tip #2,Second quick tip description.,['tag4'],2023-03-05 09:00:00+00:00,6000,250,0,15,PT4M0S,hd,True,None,None,None,9,6,3,818
7,id8,Channel B,Video Review,Review of something.,['review'],2023-03-10 14:00:00+00:00,35000,1800,0,100,PT25M15S,hd,False,None,None,None,14,4,3,813
8,id9,Channel A,In-depth Guide,Detailed guide.,['guide'],2023-03-15 17:00:00+00:00,90000,5000,0,300,PT50M0S,hd,True,None,None,None,17,2,3,808
9,id10,Channel C,Problem Solving,Solving a problem step-by-step.,"['problem', 'solution']",2023-03-20 13:00:00+00:00,12000,600,0,40,PT12M0S,sd,False,None,None,None,13,0,3,803


In [20]:
df.columns

Index(['video_id', 'title', 'description', 'tags', 'publishedAt', 'viewCount',
       'likeCount', 'commentCount', 'duration'],
      dtype='object')